In [1]:
import os
print(os.getcwd())
os.chdir("/Users/M1HR/Desktop/MIGRAINE")

/Users/M1HR/Desktop/MIGRAINE/code


In [11]:
pip install torch

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install transformers[torch]

In [ ]:
pip install seqeval

In [ ]:


import json
import re
import random
from pathlib import Path
import pandas as pd
import numpy as np
from collections import defaultdict

import torch
from torch.utils.data import Dataset, DataLoader
from transformers import (
    AutoTokenizer,
    AutoModelForTokenClassification,
    TrainingArguments,
    Trainer,
    DataCollatorForTokenClassification
)
from sklearn.model_selection import train_test_split
from seqeval.metrics import classification_report, f1_score
import warnings
warnings.filterwarnings('ignore')

# =============================================================================
# 1. CONFIGURATION
# =============================================================================

CONFIG = {
    'model_name': 'emilyalsentzer/Bio_ClinicalBERT',  # Pre-trained on clinical text
    'max_length': 128,  # Increased from 64 for longer contexts
    'batch_size': 8,    # Increased from 4 for better training
    'learning_rate': 2e-5,
    'num_epochs': 1,    # Increased from 1 for better learning
    'output_dir': 'models/clinical_ner',
    'data_dir': 'data/preprocessed',
    'sample_ratio': 1.0  # Use all data (was 0.2)
}


ENTITY_LABELS = [
    'O',  # Outside
    
    # Pain characteristics 
    'B-LOCATION', 'I-LOCATION',           # unilateral/bilateral
    'B-CHARACTER', 'I-CHARACTER',         # throbbing/pressing
    'B-INTENSITY', 'I-INTENSITY',         # mild/moderate/severe
    
    # Primary symptoms 
    'B-NAUSEA', 'I-NAUSEA',
    'B-VOMIT', 'I-VOMIT',
    'B-PHONOPHOBIA', 'I-PHONOPHOBIA',     # Sound sensitivity
    'B-PHOTOPHOBIA', 'I-PHOTOPHOBIA',     # Light sensitivity
    
    # Visual symptoms 
    'B-VISUAL', 'I-VISUAL',               # Visual aura
    'B-DIPLOPIA', 'I-DIPLOPIA',           # Double vision
    'B-VISUAL_DEFECT', 'I-VISUAL_DEFECT', # Visual field defect
    
    # Sensory symptoms 
    'B-SENSORY', 'I-SENSORY',
    'B-PARESTHESIA', 'I-PARESTHESIA',     # Bilateral tingling
    
    # Speech/language 
    'B-DYSPHASIA', 'I-DYSPHASIA',         # Word-finding difficulty
    'B-DYSARTHRIA', 'I-DYSARTHRIA',       # Slurred speech
    
    # Balance/coordination 
    'B-VERTIGO', 'I-VERTIGO',             # Dizziness
    'B-ATAXIA', 'I-ATAXIA',               # Coordination problems
    
    # Auditory 
    'B-TINNITUS', 'I-TINNITUS',           # Ringing in ears
    'B-HYPOACUSIS', 'I-HYPOACUSIS',       # Hearing loss
    
    # Consciousness 
    'B-CONSCIENCE', 'I-CONSCIENCE',       # Altered consciousness
    
    # Temporal/frequency 
    'B-DURATION', 'I-DURATION',           # How long
    'B-FREQUENCY', 'I-FREQUENCY',         # How often
    
    # Family history 
    'B-DPF', 'I-DPF',                     # Family history
]

# Create label mappings
# Convert labels to numbers for model training
label2id = {label: i for i, label in enumerate(ENTITY_LABELS)}
id2label = {i: label for label, i in label2id.items()}

# =============================================================================
# 2. COMPREHENSIVE ENTITY PATTERNS
# =============================================================================


# Negation words to detect absence of symptoms
NEGATION_WORDS = [
    'no', 'not', 'none', 'never', 'without',
    "didn't", "didnt", "don't", "dont", "wasn't", "wasnt",
    "weren't", "werent", "isn't", "isnt", "aren't", "arent",
    'absent', 'free', 'clear'
]

ENTITY_PATTERNS = {
    # ==========================================================================
    # LOCATION (Unilateral vs Bilateral)
    # ==========================================================================
    'LOCATION': [
        # Unilateral patterns
        r'\b(left|right)\s+(side|temple|hemisphere|forehead|half|eye)\b',
        r'\b(one|single)\s+side\b',
        r'\bhemicranial\b',
        r'\btemporal\b',
        r'\bunilateral\b',
        r'\bbehind\s+(left|right)\s+eye\b',
        r'\b(left|right)\s+half\s+of\s+head\b',
        
        # Bilateral patterns
        r'\bboth\s+(sides|temples)\b',
        r'\b(whole|entire)\s+head\b',
        r'\ball\s+over\b',
        r'\beverywhere\b',
        r'\bbilateral\b',
        r'\bgeneralized\b',
        r'\bdiffuse\b',
    ],
    
    # ==========================================================================
    # CHARACTER (Throbbing vs Pressing)
    # ==========================================================================
    'CHARACTER': [
        # Throbbing/pulsating patterns
        r'\b(throbbing|pulsating|pulsing|pounding)\b',
        r'\b(beating|hammering|drumming|banging|thumping)\b',
        r'\bpulse[-\s]?like\b',
        r'\brhythmic\b',
        r'\blike\s+a\s+heartbeat\b',
        r'\blike\s+drumming\b',
        
        # Pressing/constant patterns
        r'\b(pressing|squeezing|tight|tightness)\b',
        r'\b(pressure|vice[-\s]?like|band[-\s]?like)\b',
        r'\b(constricting|crushing|heavy|weighted)\b',
        r'\bdull\s+ache\b',
        r'\bsteady\s+ache\b',
        r'\b(constant|continuous|persistent)\b',
    ],
    
    # ==========================================================================
    # INTENSITY (Mild, Moderate, Severe)
    # ==========================================================================
    'INTENSITY': [
        # Pain scale ratings 
        r'\b[1-9]\s*/\s*10\b',  # 1/10 to 9/10 (must have /10)
        r'\b10\s*/\s*10\b',     # 10/10
        r'\b\d+\s+out\s+of\s+10\b',  # X out of 10
        
        # Mild 
        r'\b(mild|minor|slight)\b',
        r'\blight(?!\s+sensitivity)\b',  # 'light' but NOT 'light sensitivity'
        r'\b(low|weak|bearable|tolerable)\b',
        r'\b(manageable|annoying|bothersome)\b',
        
        # Moderate
        r'\b(moderate|medium|average)\b',
        r'\b(noticeable|significant)\b',
        r'\b(uncomfortable|distressing)\b',
        
        # Severe
        r'\b(severe|terrible|awful|horrible)\b',
        r'\b(intense|extreme|unbearable|excruciating)\b',
        r'\b(worst\s+ever|agonizing|debilitating)\b',
        r'\b(really\s+)?bad\b',
        r'\bvery\s+bad\b',
        r'\bsuper\s+bad\b',
        r'\bcouldn\'?t\s+function\b',
        r'\bincapacitating\b',
    ],
    
    # ==========================================================================
    # NAUSEA
    # ==========================================================================
    'NAUSEA': [
        r'\bnausea\b',
        r'\bnauseous\b',
        r'\bqueasy\b',
        r'\b(sick|feeling\s+sick|feel\s+sick)\b',
        r'\bsick\s+to\s+(my\s+)?stomach\b',
        r'\bnauseated\b',
        r'\bupset\s+stomach\b',
        r'\bwant(ed)?\s+to\s+throw\s+up\b',
        r'\bfeel\s+like\s+throwing\s+up\b',
        r'\bgonna\s+be\s+sick\b',
    ],
    
    # ==========================================================================
    # VOMITING
    # ==========================================================================
    'VOMIT': [
        r'\bvomit(ing|ed)?\b',
        r'\bthrew\s+up\b',
        r'\bthrow\s+up\b',
        r'\bpuk(ed|ing)\b',
        r'\bbeing\s+sick\b',
        r'\bgot\s+sick\b',
        r'\bretching\b',
        r'\bheaving\b',
    ],
    
    # ==========================================================================
    # PHOTOPHOBIA (Light sensitivity)
    # ==========================================================================
    'PHOTOPHOBIA': [
        r'\bphotophobia\b',
        r'\blight\s+sensitivity\b',
        r'\bsensitive\s+to\s+light\b',
        r'\b(lights?|bright\s+lights?)\s+hurt\b',
        r'\bcan\'?t\s+stand\s+(the\s+)?light\b',
        r'\beverything\s+(too\s+)?bright\b',
        r'\b(too\s+)?bright\b',
        r'\b(had\s+to\s+)?close\s+(the\s+)?(curtains|blinds)\b',
        r'\bwore\s+sunglasses\s+inside\b',
        r'\beyes\s+hurt\s+from\s+light\b',
    ],
    
    # ==========================================================================
    # PHONOPHOBIA (Sound sensitivity)
    # ==========================================================================
    'PHONOPHOBIA': [
        r'\bphonophobia\b',
        r'\b(sound|noise)\s+sensitivity\b',
        r'\bsensitive\s+to\s+(sound|noise)\b',
        r'\b(sounds?|noise)\s+hurt\b',
        r'\bcan\'?t\s+stand\s+(the\s+)?(sound|noise)\b',
        r'\beverything\s+(too\s+)?loud\b',
        r'\b(sounds?|noise)\s+(too\s+)?loud\b',
        r'\b(cover|plug)(ed)?\s+(my\s+)?ears\b',
        r'\b(had\s+to\s+)?plug\s+ears\b',
    ],
    
    # ==========================================================================
    # VISUAL AURA
    # ==========================================================================
    'VISUAL': [
        r'\bvisual\s+aura\b',
        r'\bzigzag\s+lines?\b',
        r'\bzig[-\s]?zag\s+lines?\b',
        r'\bjagged\s+lines?\b',
        r'\bflashing\s+(lights?)?\b',
        r'\bsparkles?\b',
        r'\bblind\s+spots?\b',
        r'\bscotoma\b',
        r'\bblurr?y\s+vision\b',
        r'\bshimmer(ing)?\b',
        r'\bspots\b',
        r'\bhalos?\b',
        r'\bfortification\s+spectra\b',
        r'\bscintillat(ions|ing)\b',
        r'\bwavy\s+lines?\b',
        r'\bgeometric\s+patterns?\b',
        r'\bvisual\s+disturb(ances|ing)\b',
        r'\bsaw\s+(zigzag|lights?|sparkles?|flashing)\b',
    ],
    
    # ==========================================================================
    # DIPLOPIA (Double vision)
    # ==========================================================================
    'DIPLOPIA': [
        r'\bdiplopia\b',
        r'\bdouble\s+vision\b',
        r'\bseeing\s+double\b',
        r'\btwo\s+of\s+everything\b',
        r'\bblurr?y\s+double\b',
        r'\bsplit\s+vision\b',
        r'\beyes\s+not\s+working\s+together\b',
    ],
    
    # ==========================================================================
    # VISUAL DEFECT
    # ==========================================================================
    'VISUAL_DEFECT': [
        r'\bvisual\s+field\s+defect\b',
        r'\bfrontal\s+(and\s+)?nasal\s+defect\b',
        r'\bfield\s+defect\b',
        r'\btunnel\s+vision\b',
        r'\bperipheral\s+vision\s+(gone|loss)\b',
        r'\bpart\s+of\s+vision\s+missing\b',
    ],
    
    # ==========================================================================
    # SENSORY (Tingling, numbness)
    # ==========================================================================
    'SENSORY': [
        r'\bsensory\s+(aura|symptoms?)\b',
        r'\btingling\b',
        r'\bnumbness\b',
        r'\bpins\s+and\s+needles\b',
        r'\bprickling\b',
        r'\bgoing\s+numb\b',
        r'\b(arm|hand|leg|foot)\s+(went\s+)?numb\b',
        r'\bnumbness\s+spreading\b',
        r'\btingling\s+spreading\b',
        r'\bparesthesia\b',
        r'\bweird\s+sensation\b',
        r'\bloss\s+of\s+feeling\b',
    ],
    
    # ==========================================================================
    # PARESTHESIA (Bilateral)
    # ==========================================================================
    'PARESTHESIA': [
        r'\bbilateral\s+paresthesia\b',
        r'\bboth\s+sides\s+(tingling|numb)\b',
        r'\btingling\s+on\s+both\s+sides\b',
        r'\bnumbness\s+both\s+sides\b',
        r'\bpins\s+and\s+needles\s+both\s+sides\b',
        r'\bsimultaneous\s+(tingling|numbness)\b',
        r'\bboth\s+(arms|hands)\s+(tingling|numb)\b',
    ],
    
    # ==========================================================================
    # DYSPHASIA (Word-finding difficulty)
    # ==========================================================================
    'DYSPHASIA': [
        r'\bdysphasia\b',
        r'\btrouble\s+finding\s+words\b',
        r'\bcan\'?t\s+find\s+words\b',
        r'\bword[-\s]?finding\s+difficult(y|ies)\b',
        r'\bdifficult(y|ies)\s+with\s+words\b',
        r'\bcouldn\'?t\s+speak\s+properly\b',
        r'\bspeech\s+problems?\b',
        r'\bstruggling\s+to\s+talk\b',
        r'\bcouldn\'?t\s+get\s+words\s+out\b',
        r'\bwords\s+wouldn\'?t\s+come\b',
    ],
    
    # ==========================================================================
    # DYSARTHRIA (Slurred speech)
    # ==========================================================================
    'DYSARTHRIA': [
        r'\bdysarthria\b',
        r'\bslurr(ed|ing)\s+(speech|words)\b',
        r'\bdifficult(y|ies)\s+speaking\b',
        r'\bcan\'?t\s+speak\s+clearly\b',
        r'\bmumbling\b',
        r'\bgarbled\s+speech\b',
        r'\btongue\s+felt\s+thick\b',
        r'\bmouth\s+didn\'?t\s+work\s+right\b',
        r'\bspeech\s+sounded\s+drunk\b',
    ],
    
    # ==========================================================================
    # VERTIGO (Dizziness)
    # ==========================================================================
    'VERTIGO': [
        r'\bvertigo\b',
        r'\bdizz(y|iness)\b',
        r'\broom\s+spinning\b',
        r'\bspinning\b',
        r'\beverything\s+spinning\b',
        r'\blightheaded\b',
        r'\blight[-\s]?headed\b',
        r'\boff\s+balance\b',
        r'\blost\s+balance\b',
        r'\bfelt\s+like\s+falling\b',
        r'\bunsteady\b',
        r'\bwobbly\b',
    ],
    
    # ==========================================================================
    # ATAXIA (Coordination problems)
    # ==========================================================================
    'ATAXIA': [
        r'\bataxia\b',
        r'\bcoordination\s+problems?\b',
        r'\black\s+of\s+coordination\b',
        r'\bclumsy\b',
        r'\buncoordinated\b',
        r'\bbalance\s+issues?\b',
        r'\bcan\'?t\s+walk\s+straight\b',
        r'\bstumbling\b',
        r'\btripping\b',
        r'\bmovements?\s+jerky\b',
        r'\bmuscle\s+control\s+problems?\b',
    ],
    
    # ==========================================================================
    # TINNITUS (Ringing in ears)
    # ==========================================================================
    'TINNITUS': [
        r'\btinnitus\b',
        r'\bringing(ing)?\s+in\s+(the\s+)?ears?\b',
        r'\bears?\s+ringing\b',
        r'\bbuzzing\s+in\s+ears?\b',
        r'\bhigh[-\s]?pitched\s+sound\b',
        r'\bwhistling\s+in\s+ears?\b',
        r'\bhumming\s+in\s+ears?\b',
        r'\bnoise\s+in\s+ears?\b',
    ],
    
    # ==========================================================================
    # HYPOACUSIS (Hearing loss)
    # ==========================================================================
    'HYPOACUSIS': [
        r'\bhypoacusis\b',
        r'\bhearing\s+loss\b',
        r'\bcan\'?t\s+hear(\s+well)?\b',
        r'\bmuffled\s+hearing\b',
        r'\bsounds?\s+muffled\b',
        r'\bhard\s+to\s+hear\b',
        r'\bears?\s+felt\s+(blocked|full)\b',
        r'\blike\s+underwater\b',
        r'\bhearing\s+underwater\b',
    ],
    
    # ==========================================================================
    # CONSCIENCE (Altered consciousness)
    # ==========================================================================
    'CONSCIENCE': [
        r'\baltered\s+consciousness\b',
        r'\bconfused\b',
        r'\bconfusion\b',
        r'\bdisoriented\b',
        r'\bout\s+of\s+it\b',
        r'\bnot\s+with\s+it\b',
        r'\bspaced\s+out\b',
        r'\b(brain\s+)?fog(gy)?\b',
        r'\bmental\s+fog\b',
        r'\bcan\'?t\s+think\s+(straight|clearly)\b',
        r'\bdazed\b',
        r'\bin\s+a\s+daze\b',
        r'\bfelt\s+(weird|strange)\b',
        r'\b(disconnected|detached)\b',
        r'\bjeopardized\s+conscience\b',
    ],
    
    # ==========================================================================
    # DURATION (How long episodes last)
    # ==========================================================================
    'DURATION': [
        r'\blasted\s+\d+\s+(hour|hr|day)s?\b',
        r'\bduration\s*:?\s*\d+\s+(hour|hr|day)s?\b',
        r'\b\d+\s+(hour|hr|day)s?\s+(long|episode)\b',
        r'\ball\s+day\b',
        r'\bentire\s+day\b',
        r'\bfor\s+\d+\s+(hour|hr|day)s?\b',
        r'\babout\s+\d+\s+(hour|hr|day)s?\b',
        r'\b(been|being)\s+(like\s+this\s+)?for\s+\d+\s+(hour|hr|day)s?\b',  #"been like this for 3 hours"
        r'\bgoing\s+on\s+\d+\s+(hour|hr|day)s?\b',  # "going on 3 hours"
        r'\b\d+\s+(hour|hr|day)s?\s+now\b',  # "3 hours now"
    ],
    
    # ==========================================================================
    # FREQUENCY (How often episodes occur)
    # ==========================================================================
    'FREQUENCY': [
        r'\b\d+\s+(times?|episodes?)\s+(per|a)\s+(month|week)\b',
        r'\bfrequency\s*:?\s*\d+\b',
        r'\bget\s+them\s+\d+\s+times?\b',
        r'\b\d+x\s+(per|a)\s+(month|week)\b',
        r'\bevery\s+(few\s+)?(day|week)s?\b',
    ],
    
    # ==========================================================================
    # DPF (Family history)
    # ==========================================================================
    'DPF': [
        r'\bfamily\s+history\b',
        r'\bruns\s+in\s+(the\s+)?family\b',
        r'\b(mom|mother|dad|father|parent)s?\s+(gets?|has|had)\s+(migraines?|headaches?|them)\b',
        r'\bfamilial\b',
        r'\bgenetic\b',
    ],
}


# =============================================================================
# 3. NEGATION-AWARE ENTITY EXTRACTION
# =============================================================================

def is_negated(text, match_start, match_end):
    
    # Get text before the entity (up to 30 characters)
    context_window = text[max(0, match_start - 30):match_start].lower()
    
    # Check for negation words
    for neg_word in NEGATION_WORDS:
        # Look for negation word followed by optional words then the entity up to 5 words apart
        pattern = rf'\b{neg_word}\b.{{0,30}}'  # Within 30 characters
        if re.search(pattern, context_window):
            return True
    
    return False


def extract_entities_from_text(text):

    text_lower = text.lower()
    tokens = text.split()  # Keep original case for tokens
    labels = ['O'] * len(tokens)  # Initialize all as Outside
    
    # Track token positions in original text
    token_positions = []
    current_pos = 0
    for token in tokens:
        start = text_lower.find(token.lower(), current_pos)
        end = start + len(token)
        token_positions.append((start, end))
        current_pos = end
    
    # For each entity type, find matches and label tokens
    for entity_type, patterns in ENTITY_PATTERNS.items():
        for pattern in patterns:
            for match in re.finditer(pattern, text_lower, re.IGNORECASE):
                match_start = match.start()
                match_end = match.end()
                
                # CHECK FOR NEGATION
                if is_negated(text_lower, match_start, match_end):
                    continue
                
                # Find which tokens this match covers
                is_first = True
                for i, (tok_start, tok_end) in enumerate(token_positions):
                    # Token overlaps with match
                    if tok_start < match_end and tok_end > match_start:
                        # Only label if not already labeled
                        if labels[i] == 'O':
                            if is_first:
                                labels[i] = f'B-{entity_type}'
                                is_first = False
                            else:
                                labels[i] = f'I-{entity_type}'
    
    return list(zip(tokens, labels))


# =============================================================================
# 4. DATASET CLASS
# =============================================================================

class NERDataset(Dataset):

    
    def __init__(self, texts, tokenizer, max_length=128):
        self.texts = texts
        self.tokenizer = tokenizer
        self.max_length = max_length
    
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        text = self.texts[idx]
        
        # Extract entities using patterns 
        token_label_pairs = extract_entities_from_text(text)
        
        if not token_label_pairs:
            # Empty text, return dummy example
            return {
                'input_ids': torch.zeros(self.max_length, dtype=torch.long),
                'attention_mask': torch.zeros(self.max_length, dtype=torch.long),
                'labels': torch.full((self.max_length,), -100, dtype=torch.long),
            }
        
        tokens, labels = zip(*token_label_pairs)
        
        # Tokenize with alignment
        encoding = self.tokenizer(
            list(tokens),
            is_split_into_words=True,  # We already split into words
            padding='max_length',
            truncation=True,
            max_length=self.max_length,
            return_tensors='pt'
        )
        
        # Align labels with subword tokens
        word_ids = encoding.word_ids(batch_index=0)
        aligned_labels = []
        
        previous_word_idx = None
        for word_idx in word_ids:
            if word_idx is None:
                # Special tokens ([CLS], [SEP], [PAD])
                aligned_labels.append(-100)  # -100 = ignore in loss
            elif word_idx != previous_word_idx:
                # First subword of a word, use original label
                aligned_labels.append(label2id[labels[word_idx]])
            else:
                # Continuation subword 
                label = labels[word_idx]
                if label.startswith('B-'):
                    label = label.replace('B-', 'I-')
                aligned_labels.append(label2id[label])
            previous_word_idx = word_idx
        
        return {
            'input_ids': encoding['input_ids'].squeeze(0),
            'attention_mask': encoding['attention_mask'].squeeze(0),
            'labels': torch.tensor(aligned_labels, dtype=torch.long),
        }


# =============================================================================
# 5. LOAD DATA
# =============================================================================

def load_preprocessed_data(data_dir, sample_ratio=1.0):
    
    data_dir = Path(data_dir)
    all_texts = []
    
    # Load from nested structure
    for model_folder in ['llama3', 'qwen']:
        json_file = data_dir / f'{model_folder}.json'
        
        if not json_file.exists():
            print(f" File not found: {json_file}")
            continue
        
        print(f"Loading {json_file}...")
        
        with open(json_file, 'r') as f:
            data = json.load(f)
        
        # Extract clean texts
        for entry in data:
            text = entry.get('clean_text', '').strip()
            all_texts.append(text)
        
        print(f"  ✓ Loaded {len(data)} entries")
    
    # Sample data if requested
    original_count = len(all_texts)
    
    if sample_ratio < 1.0:
        random.seed(42)  # For reproducibility
        sample_size = int(len(all_texts) * sample_ratio)
        all_texts = random.sample(all_texts, sample_size)
        print(f"\nSampled {len(all_texts)}/{original_count} texts ({sample_ratio*100:.0f}%)")
    else:
        print(f"\n✓ Total texts: {len(all_texts)}")
    
    return all_texts

def extract_patient_id(filename):
    match = re.search(r'patient[_\s]*(\d+)', str(filename), re.IGNORECASE)
    if match:
        return int(match.group(1)) 
    return 0
# =============================================================================
# 6. TRAINING
# =============================================================================

def train_ner_model():
  
    print("="*80)
    print("ENHANCED MEDICAL DIARY NER TRAINING")
    print("="*80)
    print("\nNEW FEATURES:")
    print("  Comprehensive vocabulary patterns")
    print("  Negation detection (no, not, didn't)")
    print("  Better BIO tagging")
    print("  More training epochs")
    print("="*80)
    
    # 1. Load data
    print("\n1. Loading data...")
    texts = load_preprocessed_data(
        CONFIG['data_dir'],
        sample_ratio=CONFIG.get('sample_ratio', 1.0)
    )
    
 
    # 2. Split data (80% train, 20% validation)
    print("\n2. Splitting data...")
    train_texts, val_texts = train_test_split(texts, test_size=0.2, random_state=42)
    print(f"  Train: {len(train_texts)}")
    print(f"  Val: {len(val_texts)}")
    
    # 3. Load tokenizer and model
    print("\n3. Loading ClinicalBERT...")
    tokenizer = AutoTokenizer.from_pretrained(CONFIG['model_name'])
    model = AutoModelForTokenClassification.from_pretrained(
        CONFIG['model_name'],
        num_labels=len(ENTITY_LABELS),
        id2label=id2label,
        label2id=label2id
    )
    print(f"  Model loaded: {CONFIG['model_name']}")
    print(f"  Entity labels: {len(ENTITY_LABELS)}")
    
    # 4. Create datasets
    print("\n4. Creating datasets...")
    train_dataset = NERDataset(train_texts, tokenizer, CONFIG['max_length'])
    val_dataset = NERDataset(val_texts, tokenizer, CONFIG['max_length'])
    print(f"  Train dataset: {len(train_dataset)} samples")
    print(f"  Val dataset: {len(val_dataset)} samples")
    
    # 5. Training arguments
    print("\n5. Setting up training...")
    
    # Clean output directory before training
    import shutil
    output_dir = Path(CONFIG['output_dir'])
    if output_dir.exists():
        print(f"  Cleaning old training artifacts from {output_dir}...")
        shutil.rmtree(output_dir)
    output_dir.mkdir(parents=True, exist_ok=True)
    
    training_args = TrainingArguments(
        output_dir=str(output_dir),
        num_train_epochs=CONFIG['num_epochs'],
        per_device_train_batch_size=CONFIG['batch_size'],
        per_device_eval_batch_size=CONFIG['batch_size'],
        learning_rate=CONFIG['learning_rate'],
        weight_decay=0.01,
        eval_strategy='epoch',
        save_strategy='epoch',  
        save_total_limit=2,     
        load_best_model_at_end=True,
        push_to_hub=False,
        logging_steps=10,
        logging_dir=f"{CONFIG['output_dir']}/logs",
    )
    
    # Data collator (handles batching)
    data_collator = DataCollatorForTokenClassification(tokenizer)
    
    # 6. Create trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        data_collator=data_collator,
        tokenizer=tokenizer,
    )
    
    # 7. Train!
    print("\n6. Training...")
    print("="*80)
    trainer.train()
    
    # 8. Save final model
    print("\n7. Saving final model...")
    final_model_path = Path(CONFIG['output_dir']) / 'final_model'
    
    # Remove old final_model if exists
    if final_model_path.exists():
        shutil.rmtree(final_model_path)
    
    # Save model and tokenizer
    trainer.save_model(final_model_path)
    tokenizer.save_pretrained(final_model_path)
    
    print(f"  ✓ Model saved to {final_model_path}")
    
    # Clean up checkpoints (optional - keep only final model)
    print("\n8. Cleaning up checkpoints...")
    for checkpoint_dir in output_dir.glob('checkpoint-*'):
        print(f"  Removing {checkpoint_dir.name}...")
        shutil.rmtree(checkpoint_dir)
    
    print(f"\n  ✓ Final directory structure:")
    print(f"     {output_dir}/")
    print(f"     └── final_model/  ← Your trained model is here!")
    
    print("\n" + "="*80)
    print("TRAINING COMPLETE!")
    print("="*80)
    
    return trainer, tokenizer


# =============================================================================
# 7. INFERENCE / ENTITY EXTRACTION
# =============================================================================

def extract_entities(text, model, tokenizer, device='cpu'):

    # Tokenize
    inputs = tokenizer(
        text,
        return_tensors='pt',
        padding=True,
        truncation=True,
        max_length=128
    ).to(device)
    
    # Predict
    model.eval()
    with torch.no_grad():
        outputs = model(**inputs)
        predictions = torch.argmax(outputs.logits, dim=-1)
    
    # Convert predictions to labels
    tokens = tokenizer.convert_ids_to_tokens(inputs['input_ids'][0])
    predicted_labels = [id2label[p.item()] for p in predictions[0]]
    
    # Extract entities
    entities = []
    current_entity = []
    current_label = None
    
    for token, label in zip(tokens, predicted_labels):
        if token in ['[CLS]', '[SEP]', '[PAD]']:
            continue
        
        if label.startswith('B-'):
            # Save previous entity
            if current_entity:
                entities.append({
                    'text': tokenizer.convert_tokens_to_string(current_entity),
                    'label': current_label
                })
            # Start new entity
            current_entity = [token]
            current_label = label[2:]  # Remove 'B-'
        
        elif label.startswith('I-') and current_label == label[2:]:
            # Continue current entity
            current_entity.append(token)
        
        else:
            # End current entity
            if current_entity:
                entities.append({
                    'text': tokenizer.convert_tokens_to_string(current_entity),
                    'label': current_label
                })
            current_entity = []
            current_label = None
    
    # Add last entity
    if current_entity:
        entities.append({
            'text': tokenizer.convert_tokens_to_string(current_entity),
            'label': current_label
        })
    
    return entities


def extract_entities_structured(text, model, tokenizer, device='cpu'):

    
    # Get raw entities
    entities = extract_entities(text, model, tokenizer, device)
    
    # Initialize all entity fields
    structured = {
        # Pain characteristics
        'location': None,
        'character': None,
        'intensity': None,
        
        # Primary symptoms
        'nausea': 'Not found',
        'vomit': 'Not found',
        'phonophobia': 'Not found',
        'photophobia': 'Not found',
        
        # Visual
        'visual': None,
        'diplopia': 'Not found',
        'visual_defect': 'Not found',
        
        # Sensory
        'sensory': None,
        'paresthesia': 'Not found',
        
        # Speech
        'dysphasia': 'Not found',
        'dysarthria': 'Not found',
        
        # Balance
        'vertigo': 'Not found',
        'ataxia': 'Not found',
        
        # Auditory
        'tinnitus': 'Not found',
        'hypoacusis': 'Not found',
        
        # Consciousness
        'conscience': 'Not found',
        
        # Temporal
        'duration': None,
        'frequency': None,
        
        # Family
        'dpf': 'Not found',
    }
    
    # Fill in found entities
    for ent in entities:
        label = ent['label'].lower()
        text_val = ent['text'].strip()
        
        if label in structured:
            structured[label] = text_val
    
    return structured


def format_entity_output(structured_entities):
    
    output = []
    
    output.append("=== PAIN CHARACTERISTICS ===")
    output.append(f"  Location:     {structured_entities['location'] or 'Not specified'}")
    output.append(f"  Character:    {structured_entities['character'] or 'Not specified'}")
    output.append(f"  Intensity:    {structured_entities['intensity'] or 'Not specified'}")
    
    output.append("\n=== PRIMARY SYMPTOMS ===")
    output.append(f"  Nausea:       {structured_entities['nausea']}")
    output.append(f"  Vomiting:     {structured_entities['vomit']}")
    output.append(f"  Sound sens:   {structured_entities['phonophobia']}")
    output.append(f"  Light sens:   {structured_entities['photophobia']}")
    
    output.append("\n=== VISUAL SYMPTOMS ===")
    output.append(f"  Visual aura:  {structured_entities['visual'] or 'Not found'}")
    output.append(f"  Diplopia:     {structured_entities['diplopia']}")
    output.append(f"  Visual def:   {structured_entities['visual_defect']}")
    
    output.append("\n=== SENSORY SYMPTOMS ===")
    output.append(f"  Sensory:      {structured_entities['sensory'] or 'Not found'}")
    output.append(f"  Paresthesia:  {structured_entities['paresthesia']}")
    
    output.append("\n=== SPEECH/LANGUAGE ===")
    output.append(f"  Dysphasia:    {structured_entities['dysphasia']}")
    output.append(f"  Dysarthria:   {structured_entities['dysarthria']}")
    
    output.append("\n=== BALANCE & AUDITORY ===")
    output.append(f"  Vertigo:      {structured_entities['vertigo']}")
    output.append(f"  Ataxia:       {structured_entities['ataxia']}")
    output.append(f"  Tinnitus:     {structured_entities['tinnitus']}")
    output.append(f"  Hypoacusis:   {structured_entities['hypoacusis']}")
    
    output.append("\n=== OTHER ===")
    output.append(f"  Consciousness: {structured_entities['conscience']}")
    output.append(f"  Duration:      {structured_entities['duration'] or 'Not specified'}")
    output.append(f"  Frequency:     {structured_entities['frequency'] or 'Not specified'}")
    output.append(f"  Family hist:   {structured_entities['dpf']}")
    
    return '\n'.join(output)


def test_model_on_samples(model_path, num_samples=5):
    
    print("\n" + "="*80)
    print("TESTING MODEL ON SAMPLE TEXTS")
    print("="*80)
    
    # Load model
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    model = AutoModelForTokenClassification.from_pretrained(model_path)
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    model.to(device)
    
    # Sample texts (including negations!)
    samples = [
        "Woke up this morning w/ a weird feeling in my head. Started w/ zigzag lines all over my vision, and then I got a blind spot in the corner of my eye. Felt nauseous, so I grabbed a granola bar and took a short walk outside. After that, the auras went away, but I still had a headache. Ugh, my head feels like its being squeezed in a vice. Took some ibuprofen and drank water, which helped a bit. Pain is severe, and I can feel it throbbing in my left temple. Been like this for 3 hours now.",
        "Bilateral headache, constant pressure. Moderate 6/10. No nausea, no light sensitivity.",  # NEGATION TEST
        "Aura started - saw zigzag lines and felt tingling. Then severe right-sided pounding 9/10. Vomiting and phonophobia.",
        "Dizzy and room spinning. Double vision. Slurred speech. No vomiting though.",  # NEGATION TEST
        "@ 9:45am, w/ out no warnin, the throbbing started @ the back of my head & kinda moved to the left side @ 10am. nausea & sound sensitivity kicked in @ 10:15am. took some meds & rested @ 10:30am. after 2 hrs, pain went away."
    ]
    
    for i, text in enumerate(samples[:num_samples], 1):
        print(f"\n{'─'*80}")
        print(f"SAMPLE {i}")
        print(f"{'─'*80}")
        print(f"Text: {text}\n")
        
        # Get structured entities
        structured = extract_entities_structured(text, model, tokenizer, device)
        
        # Format and print
        formatted = format_entity_output(structured)
        print(formatted)


# =============================================================================
# 8. BATCH PROCESSING
# =============================================================================


def process_all_diaries(model_path, data_dir, output_file):

    
    print("\n" + "="*80)
    print("BATCH PROCESSING ALL DIARIES")
    print("="*80)
    
    # =========================================================================
    # 1. LOAD MODEL
    # =========================================================================
    print("\n1. Loading model...")
    print(f"   Model path: {model_path}")
    
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    model = AutoModelForTokenClassification.from_pretrained(model_path)
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    model.to(device)
    
    print(f"   Model loaded on {device}")
    
    # =========================================================================
    # 2. FIND DIARY FILES
    # =========================================================================
    print(f"\n2. Finding diary files in {data_dir}...")
    data_path = Path(data_dir)
    
    if not data_path.exists():
        print(f"   Directory not found: {data_path}")
        return None
    
    # Only process llama3.json and qwen.json
    target_files = ['llama3.json', 'qwen.json']
    diary_files = [data_path / f for f in target_files if (data_path / f).exists()]
    
    if not diary_files:
        print(f"   No diary files found!")
        return None
    
    print(f"   ✓ Found {len(diary_files)} files:")
    for f in diary_files:
        print(f"      - {f.name}")
    
    # =========================================================================
    # 3. PROCESS EACH FILE - NO SKIPS!
    # =========================================================================
    print("\n3. Processing diary entries (100% GUARANTEE MODE)...")
    
    results = []
    total_loaded = 0
    missing_filename_count = 0
    
    for json_file in diary_files:
        print(f"\n   📁 Processing: {json_file.name}")
        
        # Load file
        with open(json_file, 'r', encoding='utf-8') as f:
            data = json.load(f)
        
        if not isinstance(data, list):
            print(f"      Unexpected format: {type(data)}")
            continue
        
        print(f"      Entries in file: {len(data)}")
        total_loaded += len(data)
        

        for i, entry in enumerate(data):
            
            if 'filename' in entry:
                patient_id = extract_patient_id(entry['filename'])
                filename = entry['filename']
            else:
                patient_id = 0
                filename = f"unknown_entry_{len(results)}"
                missing_filename_count += 1
            
            text = entry.get('clean_text', entry.get('text', ''))
            if not text:
                text = ""  
            
            structured = extract_entities_structured(text, model, tokenizer, device)
            
            results.append({
                'text_id': len(results),
                'patient_id': patient_id,
                'filename': filename,
                'text': text[:200] if text else "",
                'entities': structured
            })
            
            # Progress indicator
            if (i + 1) % 100 == 0:
                print(f"         Processed {i+1}/{len(data)} entries...")
        
        print(f"      ✓ Completed {json_file.name}")
        print(f"      ✓ Processed ALL {len(data)} entries")
    
    # =========================================================================
    # 4. VERIFICATION
    # =========================================================================
    print(f"\n   Processing Summary:")
    print(f"      Total entries loaded:    {total_loaded}")
    print(f"      Total entries processed: {len(results)}")
    
    if len(results) == total_loaded:
        print(f"      VERIFIED: All {total_loaded} entries saved!")
    else:
        print(f"      ERROR: Mismatch!")
        print(f"         Expected: {total_loaded}")
        print(f"         Got: {len(results)}")
        print(f"         Missing: {total_loaded - len(results)}")
    
    if missing_filename_count > 0:
        print(f"       Entries with missing filename: {missing_filename_count}")
        print(f"         (These were saved with default values)")
    
    if not results:
        print("\n   No results to save!")
        return None
    
    # =========================================================================
    # 5. SAVE JSON
    # =========================================================================
    print("\n4. Saving results...")
    output_path = Path(output_file)
    output_path.parent.mkdir(parents=True, exist_ok=True)
    
    with open(output_path, 'w', encoding='utf-8') as f:
        json.dump(results, f, indent=2, ensure_ascii=False)
    
    file_size_kb = output_path.stat().st_size / 1024
    print(f"   ✓ JSON saved: {output_path}")
    print(f"      Size: {file_size_kb:.1f} KB")
    print(f"      Entries: {len(results)}")
    
    # =========================================================================
    # 6. SAVE CSV
    # =========================================================================
    csv_path = output_path.with_suffix('.csv')
    
    csv_data = []
    for r in results:
        row = {
            'text_id': r['text_id'],
            'patient_id': r['patient_id'],
            'filename': r['filename'],
            **r['entities']
        }
        csv_data.append(row)
    
    df = pd.DataFrame(csv_data)
    df.to_csv(csv_path, index=False, encoding='utf-8')
    
    csv_size_kb = csv_path.stat().st_size / 1024
    print(f"   ✓ CSV saved: {csv_path}")
    print(f"      Size: {csv_size_kb:.1f} KB")
    print(f"      Rows: {len(df)}")
    
    # =========================================================================
    # 7. FINAL VERIFICATION
    # =========================================================================
    print("\n5. Final Verification:")
    
    # Re-read files to verify
    with open(output_path, 'r') as f:
        saved_json = json.load(f)
    
    saved_csv = pd.read_csv(csv_path)
    
    print(f"   JSON entries:        {len(saved_json)}")
    print(f"   CSV rows:            {len(saved_csv)}")
    print(f"   Expected:            {total_loaded}")
    
    # TRIPLE VERIFICATION
    all_match = (
        len(results) == total_loaded and
        len(saved_json) == total_loaded and
        len(saved_csv) == total_loaded
    )
    
    if all_match:
        print(f"\n   100% VERIFIED!")
        print(f"   All {total_loaded} entries saved correctly!")
        print(f"   JSON, CSV, and memory all match!")
    else:
        print(f"\n   VERIFICATION FAILED!")
        print(f"      In memory:  {len(results)}")
        print(f"      JSON file:  {len(saved_json)}")
        print(f"      CSV file:   {len(saved_csv)}")
        print(f"      Expected:   {total_loaded}")
    
    # Statistics
    print("\n6. Statistics:")
    print(f"   Files processed:     {len(diary_files)}")
    print(f"   Total entries:       {total_loaded}")
    print(f"   Entries saved:       {len(results)}")
    
    if len(results) > 0:
        print(f"   Unique patients:     {df['patient_id'].nunique()}")
        print(f"   Entries per patient: {len(results) / df['patient_id'].nunique():.1f}")
        
        # Count blank days
        blank_count = sum(1 for r in results if not r['text'] or r['text'].lower() in ['blank', ''])
        if blank_count > 0:
            print(f"   Blank days kept:     {blank_count} ")
    
    print("\n" + "="*80)
    print("BATCH PROCESSING COMPLETE!")
    print("="*80)
    print(f"\nGUARANTEE FULFILLED: {len(results)}/{total_loaded} entries saved")
    
    return results


# =============================================================================
# MAIN EXECUTION
# =============================================================================

if __name__ == "__main__":
    
    print("\n" + "="*80)    
    # Train model
    print("\nSTEP 1: Training NER model...")
    trainer, tokenizer = train_ner_model()
    
    # Test on samples
    print("\nSTEP 2: Testing on sample texts...")
    model_path = Path(CONFIG['output_dir']) / 'final_model'
    test_model_on_samples(model_path, num_samples=5)
    
    # Process all diaries
    print("\nSTEP 3: Processing all diaries ...")
    results = process_all_diaries(
        model_path=model_path,
        data_dir=CONFIG['data_dir'],
        output_file='data/ner_results/extracted_entities.json'
    )
    
    # Final summary
    print("\n" + "="*80)
    print("PIPELINE COMPLETE!")
    print("="*80)
    
    if results:
        print(f"\nSuccess!")
        print(f"   Model: {model_path}")
        print(f"   Results: data/ner_results/extracted_entities.json")
        print(f"   Total entries: {len(results)}")
        print(f"\n All entries saved - verified!")
    else:
        print(f"\n No results generated!")

In [ ]:
import re
import json
from datetime import datetime, timedelta
from pathlib import Path
import pandas as pd
from collections import defaultdict

# =============================================================================
# 1. DURATION EXTRACTION & CALCULATION
# =============================================================================

def parse_time(time_str):

    time_str = time_str.strip().lower().replace('@', '').replace(' ', '')
    
    formats = [
        '%I:%M%p',    # 10:45am
        '%I%p',       # 11pm
        '%H:%M',      # 13:30
    ]
    
    for fmt in formats:
        try:
            time_obj = datetime.strptime(time_str, fmt).time()
            return datetime.combine(datetime.today(), time_obj)
        except ValueError:
            continue
    
    return None


def calculate_duration_from_times(start_str, end_str):

    
    start = parse_time(start_str)
    end = parse_time(end_str)
    
    if not start or not end:
        return None
    
    duration = end - start
    
    # Handle overnight (end < start)
    if duration.total_seconds() < 0:
        duration += timedelta(days=1)
    
    hours = duration.total_seconds() / 3600
    return round(hours, 2)


def extract_duration_from_text(text):

    
    text_lower = text.lower()
    
    # STRATEGY 1: Explicit duration
    explicit_patterns = [
        r'lasted\s+(?:for\s+)?(\d+(?:\.\d+)?)\s+hours?',
        r'duration\s*:?\s*(\d+(?:\.\d+)?)\s+hours?',
        r'for\s+(\d+(?:\.\d+)?)\s+hours?',
        r'(\d+(?:\.\d+)?)\s+hours?\s+(?:long|total)',
        r'attack\s+lasted\s+(?:for\s+)?(\d+(?:\.\d+)?)\s+hours?',
    ]
    
    for pattern in explicit_patterns:
        match = re.search(pattern, text_lower)
        if match:
            hours = float(match.group(1))
            return {
                'duration_hours': hours,
                'duration_text': match.group(0),
                'method': 'explicit'
            }
    
    # STRATEGY 2: Time range
    # More flexible patterns for time extraction
    start_patterns = [
        r'started?\s+@?\s*(\d{1,2}:?\d{0,2}\s*(?:am|pm))',
        r'began\s+@?\s*(\d{1,2}:?\d{0,2}\s*(?:am|pm))',
    ]
    
    end_patterns = [
        r'ended?\s+@?\s*(\d{1,2}:?\d{0,2}\s*(?:am|pm))',
        r'finished\s+@?\s*(\d{1,2}:?\d{0,2}\s*(?:am|pm))',
        r'stopped\s+@?\s*(\d{1,2}:?\d{0,2}\s*(?:am|pm))',
    ]
    
    start_time = None
    end_time = None
    
    for pattern in start_patterns:
        match = re.search(pattern, text_lower)
        if match:
            start_time = match.group(1)
            break
    
    for pattern in end_patterns:
        match = re.search(pattern, text_lower)
        if match:
            end_time = match.group(1)
            break
    
    if start_time and end_time:
        hours = calculate_duration_from_times(start_time, end_time)
        if hours:
            return {
                'duration_hours': hours,
                'duration_text': f"{start_time} to {end_time}",
                'method': 'time_range'
            }
    
    # STRATEGY 3: Approximate
    approximate_patterns = {
        r'\ball\s+day\b': 12.0,
        r'\bentire\s+day\b': 12.0,
        r'\bfew\s+hours\b': 3.0,
        r'\bcouple\s+(?:of\s+)?hours\b': 2.0,
        r'\bseveral\s+hours\b': 4.0,
    }
    
    for pattern, hours in approximate_patterns.items():
        if re.search(pattern, text_lower):
            return {
                'duration_hours': hours,
                'duration_text': re.search(pattern, text_lower).group(0),
                'method': 'approximate'
            }
    
    return {
        'duration_hours': None,
        'duration_text': None,
        'method': None
    }


# =============================================================================
# 2. FREQUENCY CALCULATION
# =============================================================================

def calculate_frequency_from_diary(diary_text):
    """
    Count headache attack days in 30-day diary.
    
    Logic:
    1. Split into days
    2. Count days with headache indicators
    3. Return attacks/month
    """
    
    # Split by "Day X:"
    day_pattern = r'Day\s+(\d+):\s*([^\n]+(?:\n(?!Day\s+\d+:)[^\n]+)*)'
    days = re.findall(day_pattern, diary_text, re.MULTILINE | re.IGNORECASE)
    
    # Headache indicators
    headache_keywords = [
        r'\bha\b',
        r'\bheadache\b',
        r'\bmigraine\b',
        r'\bpain\b',
        r'\bthrobbing\b',
        r'\bpounding\b',
        r'\bpulsating\b',
        r'\baura\b',
    ]
    
    attack_days = []
    
    for day_num, day_text in days:
        text_lower = day_text.lower().strip()
        
        # Skip blank days
        if not text_lower or text_lower in ['blank', '']:
            continue
        
        # Check for headache
        has_headache = any(
            re.search(kw, text_lower)
            for kw in headache_keywords
        )
        
        if has_headache:
            attack_days.append(int(day_num))
    
    return {
        'attacks_per_month': len(attack_days),
        'attack_days': attack_days,
        'total_days': len(days),
        'method': 'diary_count'
    }


# =============================================================================
# 3. ENHANCED SYMPTOM PATTERNS
# =============================================================================

ENHANCED_PATTERNS = {
    'TINNITUS': [
        # All your original patterns
        r'\btinnitus\b',
        r'\bringing(?:ing)?\s+in\s+(?:the\s+)?ears?\b',
        r'\bears?\s+(?:were\s+)?ringing\b',
        r'\bbuzzing\s+(?:in\s+)?(?:the\s+)?ears?\b',
        r'\bhigh[-\s]?pitched\s+(?:sound|noise|ringing)\b',
        r'\bwhistling\s+(?:in\s+)?(?:the\s+)?ears?\b',
        r'\bhumming\s+(?:in\s+)?(?:the\s+)?ears?\b',
        r'\bnoise\s+in\s+(?:my\s+)?ears?\b',
        
        # ADDED: Common variations
        r'\bears?\s+(?:making|had)\s+(?:a\s+)?(?:ringing|buzzing|humming)\b',
        r'\bheard?\s+(?:ringing|buzzing|humming)\b',
        r'\bears?\s+(?:kept|started)\s+(?:ringing|buzzing)\b',
        r'\bringing\s+(?:sound|noise)\b',
        r'\bbuzzing\s+(?:sound|noise)\b',
        r'\bstrange\s+(?:sound|noise)\s+in\s+ears?\b',
    ],
    
    'VISUAL': [
        # Original patterns
        r'\bvisual\s+aura\b',
        r'\bzigzag\s+lines?\b',
        r'\bzig[-\s]?zag\s+lines?\b',
        r'\bjagged\s+lines?\b',
        r'\bflashing\s+(?:lights?|vision)?\b',
        r'\bsparkles?\b',
        r'\bblind\s+spots?\b',
        r'\bscotoma\b',
        
        r'\bgot\s+(?:the\s+)?(?:typical|usual)\s+aura\b',
        r'\bzigzag\s+lines?\s+&\s+blind\s+spots?\b',  # "zigzag lines & blind spots"
        r'\bsaw\s+zigzag\b',
        r'\bvision\s+(?:got\s+)?(?:weird|strange|blurry)\b',
        r'\bseeing\s+(?:zigzag|lights?|sparkles?|spots?)\b',
        r'\bblind\s+spots?\s+in\s+(?:my\s+)?vision\b',
    ],
    
    'SENSORY': [
        # Original
        r'\bsensory\s+(?:aura|symptoms?)\b',
        r'\btingling\b',
        r'\bnumbness\b',
        r'\bpins\s+and\s+needles\b',
        
        r'\btingling\s+&\s+numbness\b',
        r'\btingling\s+(?:in\s+)?(?:my\s+)?(?:fingers?|toes?)\b',
        r'\bnumbness\s+(?:in\s+)?(?:my\s+)?(?:fingers?|toes?)\b',
        r'\b(?:fingers?|toes?)\s+(?:were\s+)?(?:tingling|numb)\b',
        r'\bfelt\s+tingling\b',
    ],
    
    'DYSPHASIA': [
        r'\bdysphasia\b',
        r'\btrouble\s+finding\s+words\b',
        r'\bcan\'?t\s+find\s+words\b',
        r'\bword[-\s]?finding\s+difficult',
        r'\bspeech\s+problems?\b',
        r'\bstruggling\s+to\s+talk\b',
    ],
    
    'DYSARTHRIA': [
        r'\bdysarthria\b',
        r'\bslurr(?:ed|ing)\s+(?:speech|words)\b',
        r'\bcan\'?t\s+speak\s+clearly\b',
        r'\bmumbling\b',
        r'\bgarbled\s+speech\b',
    ],
    
    'DIPLOPIA': [
        r'\bdiplopia\b',
        r'\bdouble\s+vision\b',
        r'\bseeing\s+double\b',
        r'\btwo\s+of\s+everything\b',
    ],
}


def enhanced_symptom_detection(text):
    """
    Detect symptoms using enhanced patterns.
    This runs AFTER NER to catch anything missed.
    """
    
    text_lower = text.lower()
    detected = {}
    
    for symptom, patterns in ENHANCED_PATTERNS.items():
        matches = []
        for pattern in patterns:
            for match in re.finditer(pattern, text_lower):
                matches.append(match.group(0))
        
        if matches:
            detected[symptom] = list(set(matches))  # Unique matches
    
    return detected


# =============================================================================
# 4. COMPLETE POST-PROCESSING PIPELINE
# =============================================================================

def post_process_ner_results(
    ner_results_json,
    output_file='data/ner_results/enhanced_entities.json'
):
    
    print("\n" + "="*80)
    print("POST-PROCESSING NER RESULTS")
    print("="*80)
    
    # Load results
    print("\n1. Loading NER results...")
    with open(ner_results_json, 'r') as f:
        results = json.load(f)
    print(f"  ✓ Loaded {len(results)} entries")
    
    # Process each
    print("\n2. Enhancing extraction...")
    enhanced = []
    
    for i, entry in enumerate(results):
        if (i + 1) % 100 == 0:
            print(f"  Processed {i+1}/{len(results)}...")
        
        text = entry['text']
        entities = entry['entities']
        
        # Extract duration
        duration_info = extract_duration_from_text(text)
        if duration_info['duration_hours']:
            entities['duration'] = duration_info['duration_hours']
            entities['duration_text'] = duration_info['duration_text']
            entities['duration_method'] = duration_info['method']
        
        # Enhanced symptoms
        enhanced_symptoms = enhanced_symptom_detection(text)
        patient_id = entry.get('patient_id', 0)
        
        # Merge with NER results
        for symptom, matches in enhanced_symptoms.items():
            key = symptom.lower()
            
            # If NER missed it, add from post-processing
            if not entities.get(key) or entities[key] == 'Not found':
                entities[key] = matches[0] if matches else 'Not found'
                entities[f'{key}_enhanced'] = True
        
        enhanced.append({
            'text_id': entry['text_id'],
            'patient_id': patient_id,
            'text': text,
            'entities': entities,
        })
    
    # Save
    print("\n3. Saving enhanced results...")
    output_path = Path(output_file)
    output_path.parent.mkdir(parents=True, exist_ok=True)
    
    with open(output_path, 'w') as f:
        json.dump(enhanced, f, indent=2)
    
    csv_path = output_path.with_suffix('.csv')
    # Create list of rows
    csv_data = []
    for r in enhanced:
        row = {
            'text_id': r['text_id'],
            'patient_id': r['patient_id'],
            **r['entities']
        }
        csv_data.append(row)

    # Create DataFrame and save
    df = pd.DataFrame(csv_data)
    df.to_csv(csv_path, index=False, encoding='utf-8')
    
    print(f"  ✓ JSON: {output_path}")
    print(f"  ✓ CSV: {csv_path}")
    
    print("\nPOST-PROCESSING COMPLETE!")
    return enhanced


# =============================================================================
# 5. TESTING
# =============================================================================

def test_on_your_example():
    
    print("\n" + "="*80)
    print("TESTING ON PATIENT 12 EXAMPLE")
    print("="*80)
    
    # Day 2 from your example
    day2_text = """HA!  Started @ 10:45am while at the office. Pain was severe & throbbing on the L side of my head. Dizziness & nausea set in @ 11:30am. Couldn't tolerate any light or sound. Went home & took ibuprofen @ 1pm, which helped a bit. Attack lasted for 3 hours. Tho, still feeling a bit queasy rn."""
    
    print("\nText:", day2_text)
    
    # Test duration extraction
    print("\n" + "─"*80)
    print("DURATION EXTRACTION:")
    print("─"*80)
    duration = extract_duration_from_text(day2_text)
    print(f"Duration: {duration['duration_hours']} hours")
    print(f"Text: {duration['duration_text']}")
    print(f"Method: {duration['method']}")
    
    # Test symptom detection
    print("\n" + "─"*80)
    print("ENHANCED SYMPTOM DETECTION:")
    print("─"*80)
    symptoms = enhanced_symptom_detection(day2_text)
    for symptom, matches in symptoms.items():
        print(f"{symptom}: {matches}")
    
    # Day 8 with visual aura
    print("\n" + "="*80)
    print("DAY 8 - VISUAL AURA TEST")
    print("="*80)
    
    day8_text = """Had a w/ the most severe HA this yr. Started @ 8:45am while getting ready for work. The pain was like a jackhammer, throbbing & pulsating on the L side. Got the typical aura tho: zigzag lines & blind spots in my vision. Had to cancel my day & just rest. Took ibuprofen & acetaminophen @ 10am, but it didn't help much. Lasted for 3 hours tho."""
    
    print("\nText:", day8_text)
    
    duration = extract_duration_from_text(day8_text)
    print(f"\nDuration: {duration['duration_hours']} hours")
    
    symptoms = enhanced_symptom_detection(day8_text)
    print(f"\nDetected symptoms:")
    for symptom, matches in symptoms.items():
        print(f"  {symptom}: {matches}")
    
    # Full diary frequency test
    print("\n" + "="*80)
    print("FREQUENCY CALCULATION TEST")
    print("="*80)
    
    full_diary = """
Patient 35

Day 1: 
blank

Day 2: 
today was a pretty chill day tho, didnt do much rn. woke up @ 8am, had breakfast w/ my sis @ 10am. had a big project @ work due today, so I was pretty stressed out. felt a weird kinda pressure in my head, like its just gonna burst any sec now. thats when the throbbing started @ 2pm. this time its on my left side, HA. nausea hit @ 2:30pm. took some meds @ 3pm & rested for a bit. after 2 hrs the pain subsided.

Day 3: 
blank

Day 4: 
tired rn. woke up @ 7am. didnt have any plans 4 today, so I just relaxed @ home. didnt feel like doin much tho. felt a slight headache @ 12pm. pain's not too intense rn, just a dull ache. prob just stress from work.

Day 5: 
@ 9:45am, w/ out no warnin, the throbbing started @ the back of my head & kinda moved to the left side @ 10am. nausea & sound sensitivity kicked in @ 10:15am. took some meds & rested @ 10:30am. after 2 hrs, pain went away.

Day 6: 
blank

Day 7: 
today was crazy busy @ work, had to finish 2 projects @ the same time. was pretty stressed out, but w/ the help of my coworkers, I got it all done. no headache tho. yay!

Day 8: 
@ 12:20pm, the pulsating started @ the front of my head & kinda moved to the left side @ 12:30pm. nausea hit @ 12:35pm. took some meds @ 1pm & rested for a bit. after 2 hrs, pain went away.

Day 9: 
blank

Day 10: 
today was a pretty relaxed day @ home. watched some movies & played video games. felt a slight headache @ 3pm. prob just stress from work.

Day 11: 
@ 9:20am, w/ out no warnin, the throbbing started @ the front of my head & kinda moved to the left side @ 9:30am. nausea & light sensitivity kicked in @ 9:35am. took some meds & rested @ 9:45am. after 2 hrs, pain went away.

Day 12: 
blank

Day 13: 
today was a pretty chill day, didnt do much rn. went 4 a walk @ 2pm & felt a slight headache @ 2:30pm. prob just stress from work.

Day 14: 
@ 12:40pm, the pulsating started @ the back of my head & kinda moved to the left side @ 12:50pm. nausea & sound sensitivity kicked in @ 12:55pm. took some meds @ 1pm & rested for a bit. after 2 hrs, pain went away.

Day 15: 
blank

Day 16: 
today was crazy busy @ work, had to finish 3 projects @ the same time. was pretty stressed out, but w/ the help of my coworkers, I got it all done. no headache tho. yay!

Day 17: 
@ 9:45am, w/ out no warnin, the throbbing started @ the front of my head & kinda moved to the left side @ 10am. nausea & light sensitivity kicked in @ 10:05am. took some meds & rested @ 10:15am. after 2 hrs, pain went away.

Day 18: 
blank

Day 19: 
today was a pretty relaxed day @ home. watched some movies & played video games. felt a slight headache @ 2pm. prob just stress from work.

Day 20: 
@ 12:15pm, the pulsating started @ the back of my head & kinda moved to the left side @ 12:25pm. nausea & sound sensitivity kicked in @ 12:30pm. took some meds @ 12:40pm & rested for a bit. after 2 hrs, pain went away.

Day 21: 
blank

Day 22: 
today was crazy busy @ work, had to finish 2 projects @ the same time. was pretty stressed out, but w/ the help of my coworkers, I got it all done. no headache tho. yay!

Day 23: 
@ 9:30am, w/ out no warnin, the throbbing started @ the front of my head & kinda moved to the left side @ 9:40am. nausea & light sensitivity kicked in @ 9:45am. took some meds & rested @ 9:55am. after 2 hrs, pain went away.

Day 24: 
blank

Day 25: 
today was a pretty relaxed day @ home. watched some movies & played video games. felt a slight headache @ 3pm. prob just stress from work.

Day 26: 
@ 12:10pm, the pulsating started @ the back of my head & kinda moved to the left side @ 12:20pm. nausea & sound sensitivity kicked in @ 12:25pm. took some meds @ 12:35pm & rested for a bit. after 2 hrs, pain went away.

Day 27: 
blank

Day 28: 
today was a pretty chill day, didnt do much rn. went 4 a walk @ 2pm & felt a slight headache @ 2:30pm. prob just stress from work.

Day 29: 
@ 9:50am, w/ out no warnin, the throbbing started @ the front of my head & kinda moved to the left side @ 10am. nausea & light sensitivity kicked in @ 10:05am. took some meds & rested @ 10:15am. after 2 hrs, pain went away.

Day 30: 
today was a pretty relaxed day @ home. watched some movies & played video games. felt a slight headache @ 4pm. prob just stress from work.
"""
    
    freq = calculate_frequency_from_diary(full_diary)
    print(f"\nAttacks per month: {freq['attacks_per_month']}")
    print(f"Attack days: {freq['attack_days']}")


if __name__ == "__main__":
    
    # Run tests
    test_on_your_example()
    
    print("\n" + "="*80)
    print("ALL TESTS COMPLETE!")
    print("="*80)
    
    # To use on actual NER results:
    post_process_ner_results(
         ner_results_json='data/ner_results/extracted_entities.json',
         output_file='data/ner_results/postenhanced_entities.json'
     )

In [ ]:
import json
import pandas as pd
from pathlib import Path
from collections import Counter, defaultdict

# =============================================================================
# 1. ADD DPF TO PATIENT SUMMARIES
# =============================================================================

def add_dpf_to_summaries(
    summaries_file='data/ner_results/patient_summaries_fixed.json',
    original_data='data/migraine_with_id.csv',
    output_file='data/ner_results/patient_summaries_with_dpf.json'
):
    
    print("\n" + "="*80)
    print("ADDING DPF TO PATIENT SUMMARIES")
    print("="*80)
    
    # Load patient summaries
    print("\n1. Loading patient summaries...")
    with open(summaries_file, 'r', encoding='utf-8') as f:
        summaries = json.load(f)
    
    print(f"   ✓ Loaded {len(summaries)} patient summaries")
    
    # Load original data with DPF
    print("\n2. Loading original data with DPF...")
    original_df = pd.read_csv(original_data)
    print(f"   ✓ Loaded {len(original_df)} rows")
    print(f"   ✓ Columns: {list(original_df.columns)}")
    
    # Check if DPF column exists
    if 'DPF' not in original_df.columns and 'dpf' not in original_df.columns:
        print("\n    DPF column not found! Available columns:")
        for col in original_df.columns:
            print(f"      - {col}")
        return None
    
    dpf_col = 'DPF' if 'DPF' in original_df.columns else 'dpf'
    
    # Map patient_id to DPF value
    print("\n3. Mapping DPF values to patients...")
    dpf_map = {}
    
    for idx, row in original_df.iterrows():
        patient_id = idx + 1  # Assuming 1-indexed
        dpf_value = row[dpf_col]
        dpf_map[patient_id] = dpf_value
    
    print(f"   ✓ Mapped {len(dpf_map)} DPF values")
    
    # Add DPF to each summary
    print("\n4. Adding DPF to summaries...")
    added_count = 0
    missing_count = 0
    
    for summary in summaries:
        patient_id = summary.get('patient_id')
        
        if patient_id in dpf_map:
            summary['dpf'] = dpf_map[patient_id]
            added_count += 1
        else:
            summary['dpf'] = 0  # Default: no family history
            missing_count += 1
    
    print(f"   ✓ Added DPF to {added_count} summaries")
    if missing_count > 0:
        print(f"   {missing_count} patients missing DPF (set to 0)")
    
    # Save updated summaries
    print("\n5. Saving updated summaries...")
    output_path = Path(output_file)
    output_path.parent.mkdir(parents=True, exist_ok=True)
    
    with open(output_path, 'w', encoding='utf-8') as f:
        json.dump(summaries, f, indent=2, ensure_ascii=False)
    
    print(f"   ✓ Saved to: {output_path}")
    
    # Statistics
    print("\n" + "="*80)
    print("DPF STATISTICS")
    print("="*80)
    
    dpf_values = [s.get('dpf', 0) for s in summaries]
    dpf_counts = Counter(dpf_values)
    
    print(f"\n DPF Distribution:")
    for dpf, count in sorted(dpf_counts.items()):
        pct = count / len(summaries) * 100
        print(f"   DPF={dpf}: {count:4d} ({pct:5.1f}%)")
    
    print("\nDPF ADDED TO SUMMARIES!")
    
    return summaries


# =============================================================================
# 2. ANALYZE ENTITY EXTRACTION QUALITY
# =============================================================================

def analyze_entity_quality(summaries_file='data/ner_results/patient_summaries.json'):
    
    print("\n" + "="*80)
    print("ANALYZING ENTITY EXTRACTION QUALITY")
    print("="*80)
    
    # Load summaries
    print("\n1. Loading patient summaries...")
    with open(summaries_file, 'r', encoding='utf-8') as f:
        summaries = json.load(f)
    
    print(f"   ✓ Loaded {len(summaries)} patients")
    
    # Define entity fields
    entity_fields = [
        'duration', 'intensity', 'location', 'character', 'frequency',
        'nausea', 'vomit', 'photophobia', 'phonophobia',
        'visual', 'sensory', 'dysphasia', 'dysarthria',
        'vertigo', 'tinnitus', 'hypoacusis', 'diplopia', 'ataxia', 'conscience',
        'dpf'
    ]
    
    # Count missing/present for each entity
    print("\n2. Analyzing entity extraction rates...")
    
    entity_stats = {}
    
    for field in entity_fields:
        present_count = 0
        missing_count = 0
        values = []
        
        for summary in summaries:
            value = summary.get(field)
            
            # Check if present
            if value is None or value in ['Not found', 'Not specified', 'None', '', 0]:
                missing_count += 1
            else:
                present_count += 1
                values.append(value)
        
        entity_stats[field] = {
            'present': present_count,
            'missing': missing_count,
            'rate': present_count / len(summaries) * 100,
            'sample_values': values[:5] if values else []
        }
    
    # Print statistics
    print("\n" + "="*80)
    print("ENTITY EXTRACTION RATES")
    print("="*80)
    
    print("\nCritical Entities (needed for diagnosis):")
    print("-" * 80)
    
    critical_entities = ['duration', 'intensity', 'location', 'character', 
                         'nausea', 'photophobia', 'phonophobia']
    
    for field in critical_entities:
        stats = entity_stats[field]
        print(f"   {field:20s}: {stats['present']:4d}/{len(summaries)} ({stats['rate']:5.1f}%)")
        if stats['sample_values']:
            print(f"      Sample values: {stats['sample_values'][:3]}")
    
    print("\nAura Symptoms:")
    print("-" * 80)
    
    aura_entities = ['visual', 'sensory', 'dysphasia', 'dysarthria']
    
    for field in aura_entities:
        stats = entity_stats[field]
        print(f"   {field:20s}: {stats['present']:4d}/{len(summaries)} ({stats['rate']:5.1f}%)")
    
    print("\nBrainstem Symptoms:")
    print("-" * 80)
    
    brainstem_entities = ['vertigo', 'tinnitus', 'hypoacusis', 'diplopia', 'ataxia', 'conscience']
    
    for field in brainstem_entities:
        stats = entity_stats[field]
        print(f"   {field:20s}: {stats['present']:4d}/{len(summaries)} ({stats['rate']:5.1f}%)")
    
    # ==========================================================================
    # 3. ANALYZE WHY "NO DIAGNOSIS" HAPPENS
    # ==========================================================================
    
    print("\n" + "="*80)
    print("ANALYZING 'NO DIAGNOSIS' CASES")
    print("="*80)
    
    print("\nChecking ICHD-3 1.1 criteria for each patient...")
    
    criteria_failures = defaultdict(int)
    
    for summary in summaries:
        patient_id = summary.get('patient_id')
        
        # Check Criterion B: Duration 4-72 hours
        duration = summary.get('duration')
        duration_ok = False
        if duration:
            try:
                dur_val = float(duration)
                duration_ok = 4 <= dur_val <= 72
            except:
                duration_ok = True  # Assume OK if can't parse
        else:
            duration_ok = True  # Assume OK if missing
        
        if not duration_ok:
            criteria_failures['Duration not 4-72 hours'] += 1
            continue
        
        # Check Criterion C: At least 2 pain characteristics
        pain_count = 0
        
        location = summary.get('location')
        if location and str(location).lower() not in ['not found', 'none', '']:
            if any(term in str(location).lower() for term in ['left', 'right', 'side', 'temple', 'unilateral']):
                pain_count += 1
        
        character = summary.get('character')
        if character and str(character).lower() not in ['not found', 'none', '']:
            if any(term in str(character).lower() for term in ['throb', 'puls', 'pound', 'beat']):
                pain_count += 1
        
        intensity = summary.get('intensity')
        if intensity:
            try:
                int_val = float(intensity)
                if int_val >= 2:  # Moderate or severe
                    pain_count += 1
            except:
                if any(term in str(intensity).lower() for term in ['moderate', 'severe', 'bad']):
                    pain_count += 1
        
        if pain_count < 2:
            criteria_failures[f'Only {pain_count} pain characteristics'] += 1
            continue
        
        # Check Criterion D: Migraine symptoms
        nausea = summary.get('nausea')
        vomit = summary.get('vomit')
        photophobia = summary.get('photophobia')
        phonophobia = summary.get('phonophobia')
        
        has_nausea_vomit = False
        if nausea and str(nausea) not in ['0', 'Not found', 'None', '']:
            has_nausea_vomit = True
        if vomit and str(vomit) not in ['0', 'Not found', 'None', '']:
            has_nausea_vomit = True
        
        has_both_phobias = False
        if (photophobia and str(photophobia) not in ['0', 'Not found', 'None', ''] and
            phonophobia and str(phonophobia) not in ['0', 'Not found', 'None', '']):
            has_both_phobias = True
        
        if not (has_nausea_vomit or has_both_phobias):
            criteria_failures['Missing migraine symptoms (D)'] += 1
            continue
    
    print("\n Criteria Failure Reasons:")
    print("-" * 80)
    for reason, count in sorted(criteria_failures.items(), key=lambda x: x[1], reverse=True):
        pct = count / len(summaries) * 100
        print(f"   {reason:50s}: {count:4d} ({pct:5.1f}%)")
    
    # ==========================================================================
    # 4. SAMPLE PATIENT ANALYSIS
    # ==========================================================================
    
    print("\n" + "="*80)
    print("SAMPLE PATIENT DETAILS")
    print("="*80)
    
    print("\n Patient 1 (Full Details):")
    print("-" * 80)
    patient1 = summaries[0]
    for key, value in patient1.items():
        print(f"   {key:20s}: {value}")
    
    print("\n Patient 2 (Full Details):")
    print("-" * 80)
    patient2 = summaries[1]
    for key, value in patient2.items():
        print(f"   {key:20s}: {value}")
    
    return entity_stats, criteria_failures


# =============================================================================
# MAIN
# =============================================================================

if __name__ == "__main__":
    
    # Step 1: Add DPF to summaries
    print("\n" + "="*80)
    print("STEP 1: ADD DPF TO PATIENT SUMMARIES")
    print("="*80)
    
    summaries_with_dpf = add_dpf_to_summaries(
        summaries_file='data/ner_results/patient_summaries.json',
        original_data='data/migraine_with_id.csv',
        output_file='data/ner_results/patient_summaries_with_dpf.json'
    )
    
    # Step 2: Analyze entity quality
    print("\n" + "="*80)
    print("STEP 2: ANALYZE ENTITY QUALITY")
    print("="*80)
    
    entity_stats, criteria_failures = analyze_entity_quality(
        summaries_file='data/ner_results/patient_summaries_with_dpf.json'
    )
    
    print("\n" + "="*80)
    print("ANALYSIS COMPLETE!")
    print("="*80)
    
    print("\nRECOMMENDATIONS:")
    print("-" * 80)
    print("1. Check entity extraction quality for:")
    for field, stats in entity_stats.items():
        if stats['rate'] < 50:
            print(f"   - {field}: Only {stats['rate']:.1f}% extracted")
    
    print("\n2. Main diagnosis failure reasons:")
    for reason, count in list(criteria_failures.items())[:3]:
        print(f"   - {reason}: {count} patients")
    

In [ ]:


import re
import json
import pandas as pd
from pathlib import Path
from collections import Counter, defaultdict
import numpy as np

# =============================================================================
# INTENSITY NORMALIZATION
# =============================================================================

def normalize_intensity(intensity_text):
 
    if not intensity_text or intensity_text in ['Not found', 'Not specified', 'None', '']:
        return 0
    
    text = str(intensity_text).lower()
    
    # CRITICAL: Exclude duration patterns
    if re.search(r'\b(?:last|lasted|been|for)\s+\d+[-\s]*\d*\s*(?:hour|hr)', text):
        return 0  # This is duration, not intensity!
    
    # Extract numeric rating
    rating_match = re.search(r'(\d+)\s*/\s*10', text)
    if rating_match:
        rating = int(rating_match.group(1))
        if 1 <= rating <= 4:
            return 1
        elif 5 <= rating <= 6:
            return 2
        elif 7 <= rating <= 10:
            return 3
    
    # Severe keywords
    if any(kw in text for kw in ['severe', 'terrible', 'awful', 'horrible', 'intense',
                                   'extreme', 'unbearable', 'excruciating', 'worst']):
        return 3
    
    # Moderate keywords
    if any(kw in text for kw in ['moderate', 'medium', 'average', 'bad', 'uncomfortable']):
        return 2
    
    # Mild keywords  
    if any(kw in text for kw in ['mild', 'light', 'minor', 'slight', 'tolerable']):
        return 1
    
    return 0


# =============================================================================
# DURATION EXTRACTION
# =============================================================================

def extract_duration_with_patterns(text):
    
    if not text or text in ['Not found', 'Not specified', 'None', '']:
        return None
    
    text_lower = str(text).lower()
    
    patterns = [
        r'\blasted\s+(\d+(?:\.\d+)?)\s*[-–]\s*(\d+(?:\.\d+)?)\s*(?:hour|hr)s?\b',
        r'\blasted\s+(?:for\s+)?(\d+(?:\.\d+)?)\s*(?:hour|hr)s?\b',
        r'\bduration\s*:?\s*(\d+(?:\.\d+)?)\s*(?:hour|hr)s?\b',
        r'\b(\d+(?:\.\d+)?)\s*(?:hour|hr)s?\s*(?:long|episode)\b',
        r'\bfor\s+(\d+(?:\.\d+)?)\s*(?:hour|hr)s?\b',
        r'\babout\s+(\d+(?:\.\d+)?)\s*(?:hour|hr)s?\b',
        r'\b(been|being)\s+(?:like\s+this\s+)?for\s+(\d+(?:\.\d+)?)\s*(?:hour|hr)s?\b',
        r'\bgoing\s+on\s+(\d+(?:\.\d+)?)\s*(?:hour|hr)s?\b',
        r'\b(\d+(?:\.\d+)?)\s*(?:hour|hr)s?\s+now\b',
    ]
    
    for pattern in patterns:
        match = re.search(pattern, text_lower)
        if match:
            if len(match.groups()) == 2 and match.group(2):
                start = float(match.group(1))
                end = float(match.group(2))
                return (start + end) / 2
            else:
                for g in match.groups():
                    if g and g.replace('.', '').isdigit():
                        return float(g)
    
    if re.search(r'\ball\s+day\b', text_lower):
        return 12.0
    
    return None


# =============================================================================
# FREQUENCY CALCULATION
# =============================================================================

def calculate_frequency_from_extractions(day_entities_list):
    
    attack_count = 0
    
    for day_entities in day_entities_list:
        if not day_entities:
            continue
        
        has_headache = False
        
        # Check for pain characteristics
        if day_entities.get('intensity') not in ['Not found', 'Not specified', None, '']:
            has_headache = True
        
        if day_entities.get('character') not in ['Not found', 'Not specified', None, '']:
            has_headache = True
        
        if day_entities.get('location') not in ['Not found', 'Not specified', None, '']:
            has_headache = True
        
        # Check for symptoms
        symptoms = ['nausea', 'vomit', 'photophobia', 'phonophobia', 'visual', 'sensory']
        for symptom in symptoms:
            val = day_entities.get(symptom)
            if val and val not in ['Not found', 'Not specified', None, '', '0']:
                has_headache = True
                break
        
        if has_headache:
            attack_count += 1
    
    return attack_count


# =============================================================================
# AGGREGATION HELPERS
# =============================================================================

def get_most_frequent(values):
    
    valid_values = [
        v for v in values 
        if v and v not in ['Not found', 'Not specified', 'None', '', 'variable']
    ]
    
    if not valid_values:
        return None
    
    counter = Counter(valid_values)
    return counter.most_common(1)[0][0]


def aggregate_symptom_binary(values):
    
    for v in values:
        if v and v not in ['Not found', 'Not specified', 'None', '', '0']:
            return 1
    return 0


# =============================================================================
# PATIENT-LEVEL AGGREGATION
# =============================================================================

def aggregate_patient_entities(patient_id, day_entities_list):

    
    # Collect values
    collected = defaultdict(list)
    
    for day_entities in day_entities_list:
        if not day_entities:
            continue
        
        for key, value in day_entities.items():
            collected[key].append(value)
    
    aggregated = {'patient_id': patient_id}
    
    # 1. DURATION - Extract with patterns, then average
    durations = []
    for val in collected.get('duration', []):
        dur = extract_duration_with_patterns(val)
        if dur and dur > 0:
            durations.append(dur)
    
    if durations:
        aggregated['duration'] = round(np.mean(durations), 2)
    else:
        aggregated['duration'] = None
    
    # 2. INTENSITY - Normalize then average
    intensities = []
    for val in collected.get('intensity', []):
        norm = normalize_intensity(val)
        if norm > 0:
            intensities.append(norm)
    
    if intensities:
        avg_intensity = round(np.mean(intensities), 1)
        aggregated['intensity'] = avg_intensity
        
        # Convert to text
        if avg_intensity <= 1.3:
            aggregated['intensity_text'] = 'mild'
        elif avg_intensity <= 2.3:
            aggregated['intensity_text'] = 'moderate'
        else:
            aggregated['intensity_text'] = 'severe'
    else:
        aggregated['intensity'] = 0
        aggregated['intensity_text'] = 'variable'
    
    # 3. LOCATION - Most frequent
    aggregated['location'] = get_most_frequent(collected.get('location', []))
    
    # 4. CHARACTER - Most frequent
    aggregated['character'] = get_most_frequent(collected.get('character', []))
    
    # 5. FREQUENCY - Count days with headache
    aggregated['frequency'] = calculate_frequency_from_extractions(day_entities_list)
    
    # 6. SYMPTOMS - Binary (1 if ever present, 0 otherwise)
    symptom_fields = [
        'nausea', 'vomit', 'photophobia', 'phonophobia',
        'visual', 'sensory', 'dysphasia', 'dysarthria',
        'vertigo', 'tinnitus', 'hypoacusis', 'diplopia',
        'ataxia', 'conscience', 'visual_defect', 'paresthesia'
    ]
    
    for symptom in symptom_fields:
        aggregated[symptom] = aggregate_symptom_binary(collected.get(symptom, []))
    
    # 7. DPF (family history)
    aggregated['dpf'] = aggregate_symptom_binary(collected.get('dpf', []))
    
    return aggregated


# =============================================================================
# BATCH PROCESSING
# =============================================================================

def aggregate_all_patients(
    ner_results_file='data/ner_results/postenhanced_entities.json',
    output_file='data/ner_results/patient_summaries.json',
    expected_patients=400
):
    
    print("\n" + "="*80)
    print("PATIENT-LEVEL ENTITY AGGREGATION")
    print("="*80)
    
    # Load day-level extractions
    print(f"\n1. Loading NER results from {ner_results_file}...")
    with open(ner_results_file, 'r', encoding='utf-8') as f:
        data = json.load(f)
    
    # Handle both list and dict formats
    if isinstance(data, list):
        day_results = data
    elif isinstance(data, dict):
        day_results = data.get('results', data.get('extractions', []))
    else:
        raise ValueError(f"Unexpected data format: {type(data)}")
    
    print(f"  Loaded {len(day_results)} day-level extractions")
    
    # Check if patient_id exists
    if day_results and 'patient_id' not in day_results[0]:
        print(f"  ERROR: 'patient_id' field not found in entries!")
        print(f"     Available fields: {list(day_results[0].keys())}")
        return None
    
    # Group by patient ID
    print("\n2. Grouping days by patient_id...")
    
    patient_groups = defaultdict(list)
    missing_patient_id = 0
    
    for entry in day_results:
        # Get patient_id directly from entry
        patient_id = entry.get('patient_id')
        
        if patient_id is None or patient_id == 0:
            missing_patient_id += 1
            continue
        
        # Get entities
        entities = entry.get('entities', {})
        
        # Add to patient group
        patient_groups[patient_id].append(entities)
    
    num_patients = len(patient_groups)
    
    if num_patients == 0:
        print(f"   ERROR: No valid patients found!")
        print(f"     Entries with missing/invalid patient_id: {missing_patient_id}")
        return None
    
    # Calculate statistics
    days_per_patient = [len(days) for days in patient_groups.values()]
    avg_days = np.mean(days_per_patient)
    min_days = np.min(days_per_patient)
    max_days = np.max(days_per_patient)
    
    print(f"  ✓ Grouped into {num_patients} patients")
    print(f"  ℹ️  Days per patient: avg={avg_days:.1f}, min={min_days}, max={max_days}")
    print(f"  ℹ️  Expected: {expected_patients} patients")
    
    if missing_patient_id > 0:
        print(f"    Skipped {missing_patient_id} entries (missing/invalid patient_id)")
    
    # Aggregate each patient
    print("\n3. Aggregating entities for each patient...")
    
    patient_summaries = []
    
    for patient_id in sorted(patient_groups.keys()):
        day_entities = patient_groups[patient_id]
        
        summary = aggregate_patient_entities(
            patient_id=patient_id,
            day_entities_list=day_entities
        )
        
        patient_summaries.append(summary)
        
        # Progress
        if len(patient_summaries) % 50 == 0:
            print(f"     Processed {len(patient_summaries)}/{num_patients} patients...")
    
    print(f"  ✓ Aggregated {len(patient_summaries)} patient summaries")
    
    # Save results
    print("\n4. Saving patient-level summaries...")
    
    output_path = Path(output_file)
    output_path.parent.mkdir(parents=True, exist_ok=True)
    
    # JSON
    with open(output_path, 'w', encoding='utf-8') as f:
        json.dump(patient_summaries, f, indent=2, ensure_ascii=False)
    
    print(f"  ✓ JSON: {output_path}")
    print(f"     Entries: {len(patient_summaries)}")
    
    # CSV
    csv_path = output_path.with_suffix('.csv')
    df = pd.DataFrame(patient_summaries)
    df.to_csv(csv_path, index=False, encoding='utf-8')
    
    print(f"  ✓ CSV: {csv_path}")
    print(f"     Rows: {len(df)}")
    
    # Summary statistics
    print("\n" + "="*80)
    print("AGGREGATION SUMMARY")
    print("="*80)
    print(f"Day-level extractions:  {len(day_results)}")
    print(f"Patient summaries:      {len(patient_summaries)}")
    print(f"Days per patient (avg): {len(day_results) / len(patient_summaries):.1f}")
    
    # Show sample
    if patient_summaries:
        print("\n📊 Sample Patient Summary:")
        print("-" * 80)
        sample = patient_summaries[0]
        for key, value in sample.items():
            print(f"  {key:20s}: {value}")
    
    # Statistics
    print("\n📊 Aggregated Statistics:")
    print("-" * 80)
    
    print(f"Duration:")
    print(f"  Mean:    {df['duration'].mean():.2f} hours")
    print(f"  Std:     {df['duration'].std():.2f} hours")
    print(f"  Missing: {df['duration'].isna().sum()} patients")
    
    print(f"\nIntensity:")
    print(f"  Mean:         {df['intensity'].mean():.2f}")
    print(f"  Distribution: {df['intensity_text'].value_counts().to_dict()}")
    
    print(f"\nFrequency:")
    print(f"  Mean: {df['frequency'].mean():.1f} attacks/month")
    print(f"  Min:  {df['frequency'].min()}")
    print(f"  Max:  {df['frequency'].max()}")
    
    print(f"\nLocation:")
    location_counts = df['location'].value_counts()
    for loc, count in location_counts.head(5).items():
        print(f"  {loc}: {count}")
    
    print("\n✅ PATIENT-LEVEL AGGREGATION COMPLETE!")
    
    return patient_summaries


# =============================================================================
# MAIN
# =============================================================================

if __name__ == "__main__":
    
    aggregate_all_patients(
        ner_results_file='data/ner_results/postenhanced_entities.json',
        output_file='data/ner_results/patient_summaries.json',
        expected_patients=400
    )